# Mini-Project - Covid-19 Cough Audio Classification

1. Explore the dataset through code
    * How many samples does the dataset contain?
    * How many classes? How many samples per class? Show a histogram of the number of instances per class
    * Play a random sample from each class
    * Describe if/how you think the data distribution will affect training of a classifier.
    * Decide what part of the dataset to use; all, some classes, some samples. Motivate your choice

In [1]:
from matplotlib import pyplot as plt
from glob import glob
import pandas as pd
import numpy as np
import os

# Load in metadata.csv
data_path = r"metadata_compiled.csv"
data = pd.read_csv(data_path, sep=",")
#data.columns = data.columns.str.replace(" ", "")

data = data[["uuid","cough_detected","SNR","age","gender","status"]].loc[data['cough_detected'] >= 0.5].dropna()
print(data["status"].value_counts())
print("Total samples",len(data))

#data.to_csv("status_check.csv")

#plt.figure(figsize=(6,4))
#plt.title("Histogram of Patient Status")
#plt.bar(data['status'].value_counts().index, data['status'].value_counts())
#plt.xticks(rotation=20, ha='right', fontsize=8)
#plt.xlabel('Class', fontsize=8)
#plt.ylabel('Frequency', fontsize=8)
#plt.show()

#print(data["cough_detected"][0])


'\nfrom matplotlib import pyplot as plt\nfrom glob import glob\nimport pandas as pd\nimport numpy as np\nimport os\n\n# Load in metadata.csv\ndata_path = r"metadata_compiled.csv"\ndata = pd.read_csv(data_path, sep=",")\n#data.columns = data.columns.str.replace(" ", "")\n\ndata = data[["uuid","cough_detected","SNR","age","gender","status"]].loc[data[\'cough_detected\'] >= 0.5].dropna()\nprint(data["status"].value_counts())\nprint("Total samples",len(data))\n\n#data.to_csv("status_check.csv")\n\n#plt.figure(figsize=(6,4))\n#plt.title("Histogram of Patient Status")\n#plt.bar(data[\'status\'].value_counts().index, data[\'status\'].value_counts())\n#plt.xticks(rotation=20, ha=\'right\', fontsize=8)\n#plt.xlabel(\'Class\', fontsize=8)\n#plt.ylabel(\'Frequency\', fontsize=8)\n#plt.show()\n\n#print(data["cough_detected"][0])\n'

In [2]:
#print(data["uuid"][1])

data_dir_path = r"../Dataset/"
# glob(os.path.join(clip_dir, '*.txt')):
t = os.listdir(data_dir_path)

"""
#data = []
#for i in t:
#    if i.endswith(".webm") or i.endswith(".ogg"):
#        data.append(os.path.join(data_dir_path, i))

#data = pd.DataFrame(data, columns=["Links"]).to_csv("files.csv", index=False)  

#print(t := os.path.join(data_dir_path, data["uuid"][1]+".json"))

#print(t[0])
#print(data["uuid"][1])
"""

webm_data = []
ogg_data = []
c = 0
for file in data['uuid']:
    if os.path.exists(os.path.join(data_dir_path, f'{file}.webm')):
        webm_data.append(os.path.join(data_dir_path, f'{file}.webm'))
    elif os.path.exists(os.path.join(data_dir_path, f'{file}.ogg')):
        ogg_data.append(os.path.join(data_dir_path, f'{file}.ogg'))
    #file_path = os.path.join(data_dir_path, f"{path}.webm")
    #webm_data.append(file_path)
    #json_data = pd.read_json(file_path, orient="table")
    #print(json_data)
  
webm_data = pd.DataFrame(webm_data)
webm_data.to_csv("webm_data.csv")
ogg_data = pd.DataFrame(ogg_data)
ogg_data.to_csv("ogg_data.csv")


'\n\nwebm_data = []\nogg_data = []\nc = 0\nfor file in data[\'uuid\']:\n    if os.path.exists(os.path.join(data_dir_path, f\'{file}.webm\')):\n        webm_data.append(os.path.join(data_dir_path, f\'{file}.webm\'))\n    elif os.path.exists(os.path.join(data_dir_path, f\'{file}.ogg\')):\n        ogg_data.append(os.path.join(data_dir_path, f\'{file}.ogg\'))\n    #file_path = os.path.join(data_dir_path, f"{path}.webm")\n    #webm_data.append(file_path)\n    #json_data = pd.read_json(file_path, orient="table")\n    #print(json_data)\n  \nwebm_data = pd.DataFrame(webm_data)\nwebm_data.to_csv("webm_data.csv")\nogg_data = pd.DataFrame(ogg_data)\nogg_data.to_csv("ogg_data.csv")\n'